In [1]:
# Utility script

# registration

In [2]:
import numpy as np
import cv2
from math import sqrt, atan2, pi

from scipy.interpolate import RectBivariateSpline as rbs
from scipy.optimize import minimize

# from scipy.stats import pearsonr
import numba

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


target_for_ncc_optimiz

In [3]:
def similarity_transform(gt, ci_s,F = None, indices = None):    
    '''
    Perform similarity (shifts+scale+rotation) transform.
    
    Input:
        gt - geometric transformation parameters (center_x, center_y, scale, rot)
                [center_x, center_y - coordinates of CI (current image) center
                 relative to RI (reference image) grid],
                    rot in degrees.;
        ci_s - size of CI,
               suposed be squred);                  # XXX - Attension.
        F - RI-based interpolant;
        indices - indices for selected samples (for bootstrap mode,
                    in FORTRAN style).
    Output:
        ri_fragm - evaluated samples in transformed coordinates;
        xGrid, yGrid - new X and Y coordinates.
        
    Comments:
        ri_fragm, xGrid, yGrid - 1D arrays (flattened 2D image).
    '''
    
    
    center_x = gt[0]
    center_y = gt[1]
    scale = gt[2]
    rot = gt[3]
    
    
    #%% Form the initial not scaled and not rotated grid.
    ini_x = center_x - (ci_s-1)/2
    ini_y = center_y - (ci_s-1)/2

    [Y_,X_] = np.meshgrid(np.arange(ci_s), np.arange(ci_s))          
    ci_xShifts = ini_x + X_.flatten(order='F')
    ci_yShifts = ini_y + Y_.flatten(order='F')
    
    if indices is not None:        # Bootstrap mode.
        ci_xShifts = ci_xShifts[indices]
        ci_yShifts = ci_yShifts[indices]
        
    # Transformation matrix.
        # (we consider the rows as X coordinate, and columns as Y coordinate)
    tM = cv2.getRotationMatrix2D( (center_x,center_y), rot,scale)
    
    src = np.array([ci_xShifts,ci_yShifts]).T
    src_ = np.array([src])
    dst = cv2.transform(src_, tM)[0]
    
    xGrid = dst[:,0]
    yGrid = dst[:,1]
    
    if F is None:
        ri_fragm = None
    else:
        ri_fragm = F.ev(xGrid, yGrid)        # XXX - ri_fragm is 1D array.
    return ri_fragm, xGrid, yGrid


#%%
# @numba.jit
@numba.jit(nopython=True)
def my_pearson(x, y):
    '''
    My realization of Pearson correlation coefficient.
    
    Input:
        x, y - input 1D numpy arrays of equal length.
    Output:
        r - Pearson correlation coefficient.
        
    Comments:
        If the calculation results in division by zero, then r = nan (float64).
        Function is designed to work effectively with numba JIT.
        Don't work when x or y are lists.
    '''
    
    n = x.size
    n = y.size   # This added to make sure that x or y are numpy ndarrays.
                     # (for example, lists don't have size method).
    mean_x = 0.
    mean_y = 0.
    for k in range(n):
        mean_x = mean_x + x[k]
        mean_y = mean_y + y[k]
    mean_x = mean_x / n
    mean_y = mean_y / n
    
    sum_k = 0.
    dx_sum = 0.
    dy_sum = 0.
    for k in range(n):
        sum_k = sum_k + (x[k]-mean_x)*(y[k]-mean_y)
        dx_sum = dx_sum + (x[k]-mean_x)**2
        dy_sum = dy_sum + (y[k]-mean_y)**2
        
    denum = np.sqrt(dx_sum*dy_sum) 
    if denum != 0:
        r = sum_k / denum
    else:
        r = np.nan
    return r



#%%
def target_for_ncc_optimiz(x,ci,F,indices = None):
    '''
    Target function for ncc (normalized cross correlation) optimization in registration problem.
    
    Input:
        x - current geometric transformation parameters (center_x, center_y, scale, rot)
                [center_x, center_y - coordinates of ci center relative to ri grid],
                        rot in degrees.;
        ci - current image (CI),
               suposed be squred);                  # XXX - Attension.
        F - RI (reference image) based interpolant (RectBivariateSpline object);
        indices - indices for selected samples (for bootstrap mode,
                    in FORTRAN style).
    Output:
        neg_pc - ncc with minus sign.
    '''
    
    ci_s = ci.shape[0]
    ri_fragm, _, _ = similarity_transform(x,ci_s,F,indices)


    #%%
    if indices is not None:
        ci = ci.flatten(order='F')
        ci = ci[indices]
        
    # --- Very slow ---
    # (pc,_) = pearsonr(ci.flatten(order='F'),
    #                   ri_fragm.flatten(order='F'))
    
    pc = my_pearson(ci.flatten(order='F'),
                    ri_fragm.flatten(order='F') )
    
    neg_pc = -pc
    return neg_pc

ncc_optimiz

In [4]:
#%%
def ncc_optimiz(ri,ci, x0,
                indices = None):
    '''
    Realize NCC (normalized cross correlation) image registration by optimization.
    
    Input:
        ri - reference image;
        ci - current image (CI),
             suposed be squred);                  # XXX - Attension.
        x0 - initial geometric transformation parameters (center_x, center_y, scale, rot)
                [center_x, center_y - coordinates of ci center relative to ri grid],
                rot in degrees.;
        indices - indices for selected samples (for bootstrap mode,
                                                in FORTRAN style).
    Output:
        res.x - the value of the geometric transformation found as a result 
                    of optimization.
    '''
    
    # Currently only works 'nelder-mead' method.
    method_ = 'nelder-mead'  # 'nelder-mead', 'Powell','CG', 
                             # 'BFGS', 'Newton-CG', 'dogleg', 
                             # 'trust-ncg', 'trust-exact’ ', 'trust-krylov'

    
    #%%
    (n,m) = ri.shape
    F = rbs(np.arange(n), np.arange(m), ri)
    
    #%%    
    # --- DEBUG (with {'disp':True} ) ---        
    # res = minimize(target_for_ncc_optimiz, x0, method=method_,
    #               options={'disp': True},
    #               args=(ci,F,indices))
    # -----------------------
    
    res = minimize(target_for_ncc_optimiz, x0, method=method_,
                   args=(ci,F,indices))

    # --- DEBUG---
    # print('res.nit = ',res.nit)
    # print('res.nfev = ',res.nfev)
    # print()
    # -----------------------

    return res.x

transf_2stage_estim

In [5]:
# XXX - DEBUG
debug_flag = False      # True, False


#%%    Relative MinMax scaler.
def im_scaler(src,low_i,high_i):
    '''
        [--Don't used at this time--]
    Scales an image from [low_i,high_i] to [0,255] bounds.
    
    Input:
        src - source image (array-like) for scaling;
        low_i,high_i - defined initial borders.
    Output:
        dst - destination image.
    '''
    
    min_ = 0
    max_ = 255
    scale = (max_ - min_) / (high_i - low_i)
    dst = scale * (src-low_i)  + min_                                                   
    return dst


#%%

def extract__scale_rot(M):
    '''
    Extract scale and rotation parameters from transformation matrix.
    
    Input:
        M - transformation matrix (from opencv getRotationMatrix2D).
    Output:
        sc,rot - scaling and rotation parameters,
                rot in degrees.
        
    Comments:
        See the documentation for opencv (4) getRotationMatrix2D to understand 
            proposed formulas.
    '''

    sc = sqrt(M[0,0]**2 + M[1,0]**2)
    rot = atan2(-M[1,0], M[1,1])    
    rot = rot/pi*180                                                           
    return (sc,rot)


#%%
def keypoint_estim(ri,ci, kp_type = 'sift',ri_kp_des = None):
    """
    Evaluate similarity transformation parameters by keypoint-based registration.
    
    Input:
        ri - reference image;
        ci - current image;
        kp_type - detector and descriptor type
                    (possible values: 'sift', 'kaze', 'akaze')
                    [ri_kp_des if defined should match kp_type];
        ri_kp_des - ri keypoints coordinates and descriptors 
                    (if ri_kp_des defined in input it will not evaluated in function).
    Output:
        c_end_x, c_end_y, sc, rot - estimated transformation parameters
                                    (center_x, center_y, scale, rotation;
                                     center_x, center_y - coordinates of ci center retive to ri),
                            rot in degrees.
        
    Comments:
        If registration fail return:  (0,0,1,0).
        At the moment the function is designed to work with kp_type == 'sift'
            (other possibilities don't checked).
    """

    if kp_type == 'sift':
        # kp_obj = cv2.xfeatures2d.SIFT_create()
        kp_obj = cv2.SIFT_create()
    elif kp_type == 'kaze':
        kp_obj = cv2.KAZE_create()
    elif kp_type == 'akaze':
        kp_obj = cv2.AKAZE_create()
    else:
        raise ValueError('Wrong value for kp_obj')
    

    if ri_kp_des is None:
        # Scale ri to [0,255] range.
        ri_scaled = cv2.normalize(ri, None, alpha=0, beta=255, 
                                  norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        
        ri_kp, ri_des = kp_obj.detectAndCompute(ri_scaled, None) 
    else:
        ri_kp, ri_des = ri_kp_des

    ci_scaled = cv2.normalize(ci, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)         
    ci_kp, ci_des = kp_obj.detectAndCompute(ci_scaled, None)

    # --- DEBUG+ ---
    # print()
    # print('--- ci_kp[ind_].pt ---')
    # for ind_ in range(len(ci_kp)):
    #     print(ci_kp[ind_].pt)
    # print()
    # -------------


    # XXX - Alternative normalization methhod (normalize ci as ri).      
# =============================================================================
#     low_i = np.amin(ri)
#     high_i = np.amax(ri)
#     
#     ci = im_scaler(ci,low_i,high_i)
#     ci[ci > 255] = 255
#     ci[ci < 0] = 0
#     ci_scaled = ci.astype(np.uint8)
#     ci_kp, ci_des = kp_obj.detectAndCompute(ci_scaled, None)
# =============================================================================
    
    if debug_flag:
        print()
        print('nRI kp = ', len(ri_kp))
        print('nCI kp = ', len(ci_kp))
        
    if (ri_des is None) or (ci_des is None):
        
        if debug_flag:
            print('Couldn\'t calculate descriptors for ri or ci')
        
        return (0,0,1,0)       
                                              

    #%% 
    # --- FLANN-based match ---
    # FLANN_INDEX_KDTREE = 1
    # index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    # search_params = dict(checks = 50)
    # flann = cv2.FlannBasedMatcher(index_params, search_params)
    # matches = flann.knnMatch(ci_des, ri_des, k=2)
    
    # --- BRUTEFORCE-based match ---
    matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_BRUTEFORCE)
    matches = matcher.knnMatch(ci_des, ri_des, 2)

    if debug_flag:
        print('Number of matches: ',len(matches))
    
    
    #%% Use Lowe's ratio test.
    good = []
    coef_ = 0.95            # XXX: 0.95 - for MRRS_2020, 0.9 - for ISDMCI_2020.
    try:
        for m,n in matches:
            if m.distance < coef_ * n.distance:                                       
                good.append(m)
                
    except ValueError:        # Handle if ri_kp has only 1 keypoint.
        # --- DEBUG ---
        # print()
        # print( 'Exception in "for m,n in matches" (transf_2stage_estim/keypoint_estim)' )
        # print('ri_kp = ', ri_kp)
        # print('ci_kp = ', ci_kp)
        # print()
        # --------------
        return (0,0,1,0) 

    if debug_flag:
        print('Len good = ', len(good))
    
    
    #%% Transformation matrix evaluation. 
    MIN_MATCH_COUNT = 6
    if len(good) >= MIN_MATCH_COUNT:
        src_pts = np.float32([ ci_kp[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ ri_kp[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

      
        # --- DEBUG+  visualize keyppoints) ---
        # print('ci.shape = ', ci.shape)
        # nKp = 7      # Number of depicted keypoints.
        # src_pts = src_pts[:nKp,:,:]
        # print('src_pts = ', src_pts)
        # print()
        # ci_kp = [ ci_kp[m.queryIdx] for m in good ]   # Sort keypoints.
        # ci_kp = ci_kp[:nKp]    

        # img_with_kp = cv2.drawKeypoints(ci.astype('uint8'),
        #                                 ci_kp, outImage = None)
        # cv2.imwrite('im_with_keypoints.jpg',img_with_kp)
        # raise NameError('Stop execution to see the keypoints')
        # ----------------------------
        
        
        # Find transformation FROM src_pts (CI) points TO dst_pts (RI) points
        #    (in CI grid origin is assumed at top-left corner; 
        #     this differs from our setup, that the origin is in the center of the CI,
        #     we take this into account when calculating the transformation parameters ).
        M, inlier = cv2.estimateAffinePartial2D(src_pts, dst_pts)      
        
    else:
        if debug_flag:
            print("Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT))
            
        return (0,0,1,0)     
    
    
    #%%
    # Calculate scale, rotation. 
    if M is not None:
        (sc,rot) = extract__scale_rot(M)
        # Since cv2.getRotationMatrix2D (with which we model trasformation in make_radPattern_im function) 
        #     and (cv2.estimateAffinePartial2D which we used to estimate transformation) use
        #     different convension (for rotation direction) we should change the sign.
        rot = -rot    
    else:   
        return (0,0,1,0)
    
    
    #%% Estimate CI center position.
    (n_ci, m_ci) = ci.shape
    c_x_ini = n_ci/2-0.5
    c_y_ini = m_ci/2-0.5
    
    c_ini = np.array([ [c_y_ini],
                       [c_x_ini],
                       [1] ])
    c_end = np.dot(M, c_ini)
    c_end_x = c_end[1]
    c_end_y = c_end[0]
    
    # Convert to scalars.
    c_end_x = c_end_x[0]
    c_end_y = c_end_y[0]
   
    return (c_end_x, c_end_y, sc, rot)
    
    
    
#%%
def transf_2stage_estim(ri,ci,
                        indices = None,
                        ri_kp_des = None,
                        true_gt_tuple = None):
    '''
    Estimate similarity transformation parameters by 2-stage registration.
    
    Input:
        ri - reference image;
        ci - current image;
        indices - indices for bootstrap (in FORTRAN style);
        ri_kp_des - ri keypoints coordinates and descriptors 
                    (if ri_kp_des defined in input it will not evaluated in function);
        true_gt_tuple - true transformation values for speed-up mode (used in this function): 
                        if 1st registration step estimates bad - we don't evaluate 2nd step
                        (in real situation we don't know how bad or not 1st step estimates).
    Output:
        gt_kp - transformation parameters by 1st stage (tuple);
        gt_ncc - transformation parameters by 1st stage (ndarray or tuple).
        
    Comments:
        If indices==None  ->  without bootstrap.
    '''
    
    #%% --- 1 stage ---
    # Geometric transformation (gt) estimation by keypoints-based registration.
    gt_kp = keypoint_estim(ri,ci,ri_kp_des = ri_kp_des)

    
    #%% --- 2 stage ---
    x0 = gt_kp
    if true_gt_tuple is not None:           # For speed-up.
        # Bounds on errors for speed-up mode (relative to true values).
        sh_b = 4           
        sc_b = 0.1          
        rot_b = 10          # in degrees.
        # Alternative: sh_b = 2, sc_b = 0.05, rot_b = 5 (as in Kybic_bs and functions.statistic.monteCarlo_reg)
        
        bound4_tuple = (sh_b, sh_b, sc_b, rot_b)
        bound4_array = np.array(bound4_tuple)
        
        true_gt_array = np.array(true_gt_tuple)
        gt_kp_array = np.array(gt_kp)
        
        true_abs_err4 = np.absolute( gt_kp_array - true_gt_array )
        if np.all(true_abs_err4 < bound4_array):
            gt_ncc = ncc_optimiz(ri,ci, x0,
                                 indices)
        else:
            gt_ncc = (-100,-100,-100,-100)
    else:
        gt_ncc = ncc_optimiz(ri,ci, x0,
                             indices)
    
    return gt_kp, gt_ncc